In [31]:
import os
import numpy as np
import json
import pandas as pd
import plotly.express as px
import datetime

from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Add, ReLU, Lambda
from tensorflow.keras.models import Model

AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = (128, 128)
EMBED_DIM = 128
BATCH_SIZE = 100
EPOCHS = 150
TEMPERATURE = 0.05


In [2]:
def contar_archivos_en_carpetas(directorio):
    # Recorre todas las carpetas dentro del directorio
    for carpeta in os.listdir(directorio):
        ruta_carpeta = os.path.join(directorio, carpeta)
        if os.path.isdir(ruta_carpeta):
            # Cuenta solo archivos (no subcarpetas)
            archivos = [f for f in os.listdir(ruta_carpeta) 
                        if os.path.isfile(os.path.join(ruta_carpeta, f))]
            print(f"Carpeta: {carpeta} -> {len(archivos)} archivos")

# Ejemplo de uso
directorio_base = "/kaggle/input/hampreprocessed/malignas_classes/train"  # Cambia esto por tu ruta
contar_archivos_en_carpetas(directorio_base)


Carpeta: mel -> 1002 archivos
Carpeta: akiec -> 296 archivos
Carpeta: bcc -> 461 archivos


In [3]:
def contar_archivos_por_clase(directorio_base):
    clases_totales = {}  # acumulador por clase

    for conjunto in ["train", "test"]:
        ruta_conjunto = os.path.join(directorio_base, conjunto)
        if not os.path.exists(ruta_conjunto):
            print(f"No existe la carpeta: {ruta_conjunto}")
            continue

        print(f"\nConjunto: {conjunto}")
        for carpeta in os.listdir(ruta_conjunto):
            ruta_carpeta = os.path.join(ruta_conjunto, carpeta)
            if os.path.isdir(ruta_carpeta):
                archivos = [f for f in os.listdir(ruta_carpeta) 
                            if os.path.isfile(os.path.join(ruta_carpeta, f))]
                cantidad = len(archivos)
                print(f"  Carpeta: {carpeta} -> {cantidad} archivos")

                # acumular por clase
                if carpeta not in clases_totales:
                    clases_totales[carpeta] = 0
                clases_totales[carpeta] += cantidad

    # Mostrar suma total por clase
    print("\nSuma total por clase (train + test):")
    for clase, total in clases_totales.items():
        print(f"  {clase} -> {total} archivos")

# Ejemplo de uso
directorio_base = "/kaggle/input/hampreprocessed/processed"  # Ruta base que contiene train y test
contar_archivos_por_clase(directorio_base)



Conjunto: train
  Carpeta: benignas -> 7254 archivos
  Carpeta: malignas -> 1759 archivos

Conjunto: test
  Carpeta: benignas -> 807 archivos
  Carpeta: malignas -> 195 archivos

Suma total por clase (train + test):
  benignas -> 8061 archivos
  malignas -> 1954 archivos


## Importación de datos

In [4]:
data_dir = "/kaggle/input/hampreprocessed/malignas_classes/train"

def get_generators(data_dir, preprocess_fn, target_size=IMG_SIZE, batch_size=BATCH_SIZE, validation_split=0.15):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_fn,
        rotation_range=60,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.12,
        brightness_range=[0.8, 1.2],
        shear_range=0.2,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=validation_split
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_generator, val_generator

In [5]:
train_generator, val_generator = get_generators(data_dir, lambda x: x/255.)
num_classes = len(train_generator.class_indices)
class_names = list(train_generator.class_indices.keys())

Found 1496 images belonging to 3 classes.
Found 263 images belonging to 3 classes.


## Modelo generador de embeddings

In [6]:
def contrastive_encoder(input_shape=(IMG_SIZE[0],IMG_SIZE[0],3), embedding_dim=EMBED_DIM):
    inputs = Input(shape=input_shape)

    # Bloque 1
    x = Conv2D(64, 3, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(64, 3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    s = Conv2D(64, 1, padding='same', use_bias=False)(inputs)
    s = BatchNormalization()(s)
    x = Add()([x, s])
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    # Bloque 2
    y = Conv2D(128, 3, padding='same', use_bias=False)(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv2D(128, 3, padding='same', use_bias=False)(y)
    y = BatchNormalization()(y)
    s2 = Conv2D(128, 1, padding='same', use_bias=False)(x)
    s2 = BatchNormalization()(s2)
    y = Add()([y, s2])
    y = ReLU()(y)
    y = MaxPooling2D()(y)

    # Bloque 3
    z = Conv2D(256, 3, padding='same', use_bias=False)(y)
    z = BatchNormalization()(z)
    z = ReLU()(z)
    z = Conv2D(256, 3, padding='same', use_bias=False)(z)
    z = BatchNormalization()(z)
    s3 = Conv2D(256, 1, padding='same', use_bias=False)(y)
    s3 = BatchNormalization()(s3)
    z = Add()([z, s3])
    z = ReLU()(z)

    z = GlobalAveragePooling2D()(z)
    z = Dense(512, activation='relu')(z)
    z = BatchNormalization()(z)

    # Proyección (cabeza contrastiva)timestamps
    p = Dense(EMBED_DIM, activation='relu')(z)
    p = Dense(EMBED_DIM)(p)
    outputs = Lambda(lambda t: tf.math.l2_normalize(t, axis=1), name="proj_norm")(p)

    return Model(inputs, outputs, name="ContrastiveEncoder")


In [7]:
class SupConLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=0.1, name="supcon"):
        super().__init__(name=name)
        self.temperature = temperature

    def call(self, y_true, features):
        """
        SupConLoss implementation.
        Args:
            y_true: [batch] integer class labels (not one-hot).
            features: [batch, dim] embeddings.
        """
        # Normalize embeddings
        features = tf.math.l2_normalize(features, axis=1)
        batch_size = tf.shape(features)[0]

        # Similarity matrix
        sim = tf.matmul(features, features, transpose_b=True)  # [B, B]
        sim = sim / self.temperature

        # Ensure labels are integers, not one-hot
        if y_true.shape.ndims > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)

        labels = tf.reshape(y_true, [-1, 1])  # [B, 1]
        mask = tf.equal(labels, tf.transpose(labels))  # [B, B]
        mask = tf.cast(mask, tf.float32)

        # Remove self-contrast
        eye = tf.eye(batch_size, dtype=tf.float32)
        logits_mask = tf.ones_like(mask) - eye
        mask = mask * logits_mask

        # Log-softmax denominator excluding self
        sim_max = tf.reduce_max(sim, axis=1, keepdims=True)
        sim = sim - sim_max
        exp_sim = tf.exp(sim) * logits_mask
        denom = tf.reduce_sum(exp_sim, axis=1, keepdims=True) + 1e-9
        log_prob = sim - tf.math.log(denom)

        # Average log-prob of positives per anchor
        pos_count = tf.reduce_sum(mask, axis=1) + 1e-9
        mean_log_pos = tf.reduce_sum(mask * log_prob, axis=1) / pos_count

        loss = -tf.reduce_mean(mean_log_pos)
        return loss


## Entrenar representaciones

In [8]:
def evaluate_embeddings(model, generator, k=3, steps=50):
    """Evalúa embeddings con linear probe o k-NN."""
    all_embeds, all_labels = [], []
    for _ in range(steps):
        images, labels = next(generator)
        embeds = model(images, training=False).numpy()
        all_embeds.append(embeds)
        all_labels.append(labels)
    X = np.concatenate(all_embeds, axis=0)
    y = np.concatenate(all_labels, axis=0)

    # k-NN
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X, y)
    y_pred = clf.predict(X)
    acc = accuracy_score(y, y_pred)
    return acc

def train_supcon(model, train_generator, val_generator, loss_fn, optimizer, epochs=50):
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = val_generator.samples // val_generator.batch_size

    train_loss = tf.keras.metrics.Mean(name="train_loss")
    val_loss = tf.keras.metrics.Mean(name="val_loss")

    for epoch in range(epochs):
        train_loss.reset_state()
        val_loss.reset_state()

        # Entrenamiento
        for _ in range(steps_per_epoch):
            images, labels = next(train_generator)
            with tf.GradientTape() as tape:
                embeddings = model(images, training=True)
                loss = loss_fn(labels, embeddings)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            train_loss.update_state(loss)

        # Validación
        for _ in range(validation_steps):
            images, labels = next(val_generator)
            embeddings = model(images, training=False)
            loss = loss_fn(labels, embeddings)
            val_loss.update_state(loss)

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss.result():.4f} - Val Loss: {val_loss.result():.4f}")

        # Evaluación cada 5 epochs
        if (epoch + 1) % 5 == 0:
            acc_knn = evaluate_embeddings(model, val_generator, k=3, steps=validation_steps)
            print(f"k-NN Acc: {acc_knn:.4f}")


In [10]:
encoder = contrastive_encoder(embedding_dim=EMBED_DIM)
encoder.trainable = True
loss_fn = SupConLoss(temperature=TEMPERATURE)
optimizer = Adam(learning_rate=8e-4)
train_supcon(encoder, train_generator, val_generator, loss_fn, optimizer, epochs=60)

I0000 00:00:1763631188.855392      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
I0000 00:00:1763631192.414032      48 cuda_dnn.cc:529] Loaded cuDNN version 90300


Epoch 1/60 - Train Loss: 6.4283 - Val Loss: 4.6105
Epoch 2/60 - Train Loss: 4.6666 - Val Loss: 4.4532
Epoch 3/60 - Train Loss: 4.6144 - Val Loss: 4.4671
Epoch 4/60 - Train Loss: 4.6048 - Val Loss: 4.6262
Epoch 5/60 - Train Loss: 4.6004 - Val Loss: 4.3693
k-NN Acc: 0.9202
Epoch 6/60 - Train Loss: 4.5983 - Val Loss: 4.5974
Epoch 7/60 - Train Loss: 4.5969 - Val Loss: 4.3624
Epoch 8/60 - Train Loss: 4.5970 - Val Loss: 4.3619
Epoch 9/60 - Train Loss: 4.5965 - Val Loss: 4.5958
Epoch 10/60 - Train Loss: 4.5952 - Val Loss: 4.3616
k-NN Acc: 0.9264
Epoch 11/60 - Train Loss: 4.5956 - Val Loss: 4.5957
Epoch 12/60 - Train Loss: 4.5946 - Val Loss: 4.3618
Epoch 13/60 - Train Loss: 4.5948 - Val Loss: 4.3618
Epoch 14/60 - Train Loss: 4.5947 - Val Loss: 4.5960
Epoch 15/60 - Train Loss: 4.5942 - Val Loss: 4.3619
k-NN Acc: 0.9264
Epoch 16/60 - Train Loss: 4.5958 - Val Loss: 4.5957
Epoch 17/60 - Train Loss: 4.5937 - Val Loss: 4.3619
Epoch 18/60 - Train Loss: 4.5941 - Val Loss: 4.3621
Epoch 19/60 - Train Lo

In [13]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

encoder.save(f"encoder_{timestamp}.keras")

## Calcular centroides de clases

In [45]:
def compute_centroids(encoder, generator):
    """
    Calcula centroides de clase a partir de un generator de Keras.
    Devuelve un dict {class_index: centroid_vector}.
    """
    embeds, labels = [], []
    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = normalize(e)  # normalizar embeddings fila a fila
        embeds.append(e)
        labels.append(np.argmax(y_batch, axis=1))  # convertir one-hot a entero
    # print(pd.Series(labels).value_counts())

    embeds = np.concatenate(embeds)
    labels = np.concatenate(labels)

    centroids = {}
    for c in np.unique(labels):
        class_embeds = embeds[labels == c]
        centroid = class_embeds.mean(axis=0)
        centroid = centroid / np.linalg.norm(centroid)  # normalizar centroide
        centroids[int(c)] = centroid.tolist()  # convertir a lista para JSON

    return centroids

def save_centroids(centroids, filename=None):
    if not filename:
        timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")
        filename = f"centroids_{timestamp}.json"
    with open(filename, "w") as f:
        json.dump(centroids, f)

def load_centroids(filename="centroids.json"):
    with open(filename, "r") as f:
        centroids = json.load(f)
    # convertir a numpy arrays
    centroids = {int(k): np.array(v) for k, v in centroids.items()}
    return centroids

In [46]:
centroids = compute_centroids(encoder, train_generator)
save_centroids(centroids)

## Evaluar

In [41]:
def predict_class(encoder, x, centroids, probs=False):
    """
    Predice la clase de una sola imagen x usando centroides.
    """
    e = encoder.predict(np.expand_dims(x, axis=0), verbose=0)
    e = normalize(e)  # normalizar embedding
    sims = {c: np.dot(e, centroids[c]) for c in centroids}
    if probs:
        return sims
    return max(sims, key=sims.get)  # clase con mayor similitud

def evaluate_accuracy(encoder, val_generator, centroids):
    """
    Calcula el accuracy del val_generator usando centroides.
    """
    y_true, y_pred = [], []

    for i in range(len(val_generator)):
        x_batch, y_batch = val_generator[i]
        labels = np.argmax(y_batch, axis=1)  # convertir one-hot a enteros

        for j in range(len(x_batch)):
            pred = predict_class(encoder, x_batch[j], centroids)
            y_true.append(labels[j])
            y_pred.append(pred)

    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy en val_generator: {acc:.4f}")
    return acc

In [42]:
evaluate_accuracy(encoder, train_generator, centroids)

Accuracy en val_generator: 0.4398


0.43983957219251335

In [39]:
evaluate_embeddings(encoder, val_generator, k=3, steps=50)

0.9357126515671471

## KNN as classifier

In [58]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def train_knn(encoder, train_generator, k=10):
    """
    Entrena un KNN sobre los embeddings del train_generator.
    Devuelve el clasificador entrenado.
    """
    X, y = [], []
    for i in range(len(train_generator)):
        x_batch, y_batch = train_generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = e / np.linalg.norm(e, axis=1, keepdims=True)  # normalizar embeddings
        X.append(e)
        y.append(np.argmax(y_batch, axis=1))

    X = np.concatenate(X)
    y = np.concatenate(y)

    knn = KNeighborsClassifier(n_neighbors=k, metric="cosine")
    knn.fit(X, y)
    return knn

def evaluate_knn(encoder, val_generator, knn):
    """
    Evalúa un KNN entrenado sobre el val_generator.
    """
    X_val, y_val = [], []
    for i in range(len(val_generator)):
        x_batch, y_batch = val_generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = e / np.linalg.norm(e, axis=1, keepdims=True)
        X_val.append(e)
        y_val.append(np.argmax(y_batch, axis=1))

    X_val = np.concatenate(X_val)
    y_val = np.concatenate(y_val)

    y_pred = knn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f"Accuracy en val_generator con KNN: {acc:.4f}")
    return acc

In [59]:
knn = train_knn(encoder, train_generator)
print("Entrenado")
evaluate_knn(encoder, val_generator, knn)

Entrenado
Accuracy en val_generator con KNN: 0.4867


0.4866920152091255

In [16]:
# encoder.trainable = False
# x = encoder.output
# clf = Dense(num_classes, activation="softmax")(x)
# classifier = Model(encoder.input, clf)
# classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [21]:
# labels = train_generator.classes
# print(pd.Series(labels).value_counts())
# print(num_classes)
# class_weights = dict(enumerate(compute_class_weight(
#     class_weight="balanced",
#     classes=np.unique(labels),
#     y=labels
# )))

# classifier.fit(
#     train_generator,
#     validation_data=val_generator,
#     epochs=100,
#     class_weight=class_weights
# )


In [22]:
# # Current timestamp
# timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

# classifier.save(f"classifier_{timestamp}.keras")

## Evaluate KNN

In [ ]:
def evaluate_knn(model, val_generator, class_names, k=5):
    embs, labs = [], []
    # recorrer todo el generador de validación
    for imgs, labels in val_generator:
        e = model(imgs, training=False).numpy()
        embs.append(e)
        # si labels es one-hot, convertir a entero con argmax
        if labels.ndim > 1:
            labs.append(np.argmax(labels, axis=1))
        else:
            labs.append(labels)

    # concatenar embeddings y etiquetas
    X = np.concatenate(embs, axis=0)
    y = np.concatenate(labs, axis=0)

    # entrenar y evaluar KNN
    knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')
    knn.fit(X, y)
    y_pred = knn.predict(X)

    print(classification_report(y, y_pred, target_names=class_names))
    print(confusion_matrix(y, y_pred))


In [ ]:
evaluate_knn(encoder, val_generator, class_names, k=3)

In [ ]:
def visualize_embeddings_3d(model, val_ds, class_names, method="tsne"):
    # 1. Extraer embeddings y etiquetas
    embs, labs = [], []
    for imgs, labels in val_ds:
        e = model(imgs, training=False).numpy()
        embs.append(e)
        labs.append(labels.numpy())
    X = np.concatenate(embs, axis=0)
    y = np.concatenate(labs, axis=0)

    # 2. Reducir a 3D
    if method == "tsne":
        reducer = TSNE(n_components=3, perplexity=30, learning_rate=200, random_state=42)
    else:
        reducer = PCA(n_components=3)
    X_reduced = reducer.fit_transform(X)

    # 3. Visualizar con Plotly
    fig = px.scatter_3d(
        x=X_reduced[:,0], y=X_reduced[:,1], z=X_reduced[:,2],
        color=[class_names[i] for i in y],
        title=f"Embeddings en 3D ({method.upper()})",
        opacity=0.7
    )
    fig.show()


In [ ]:
visualize_embeddings_3d(encoder, val_generator, class_names)